In [1]:
import snowflake.connector
import pyodbc
import pandas as pd
import os

account = 'ffa72186.us-east-1'

user = 'B042304@MERCANTIL.COM.BR'
password = 'trabalho@03T'

warehouse = 'WH_DEV'
database = 'SDX_GERENCIA_CRESCIMENTO'
schema = 'CONHECIMENTO_CLIENTE_BRONZE'
authenticator = 'externalbrowser'

conn_snow = snowflake.connector.connect(
    user=user,
    authenticator=authenticator,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema
)

cur_snow = conn_snow.cursor()

In [2]:
names = [
    'COD_CIDADE',
    'NOM_CIDADE',
    'COD_ESTADO'
]
    
    
df_cidades = pd.read_csv(f'I:\Crescimento\Temporário\Rheydne\CEP\cities.csv', sep=',', names=names)
df_cidades

,COD_CIDADE,NOM_CIDADE,COD_ESTADO
0,1,Abacate da Pedreira (Macapá),4
1,2,Abadia de Goiás,9
2,3,Abadia dos Dourados,11
3,4,Abadia (Jandaíra),5
4,5,Abadiânia,9
...,...,...,...
10669,10693,Imbassaí (Mata de São João),5
10670,10694,Morro do Forno (Morrinhos do Sul),23
10671,10695,Parati Mirim (Paraty),19
10672,10696,Baunilha (Colatina),8


In [3]:
names = [
    'COD_ESTADO',
    'ESTADO',
    'UF'
]
    
    
df_estados = pd.read_csv(f'I:\Crescimento\Temporário\Rheydne\CEP\states.csv', sep=',', names=names)
df_estados

,COD_ESTADO,ESTADO,UF
0,1,Acre,AC
1,2,Alagoas,AL
2,3,Amazonas,AM
3,4,Amapá,AP
4,5,Bahia,BA
5,6,Ceara,CE
6,7,Distrito Federal,DF
7,8,Espirito Santo,ES
8,9,Goiás,GO
9,10,Maranhão,MA


In [4]:
def ReadFiles(path, name_file):
    
    file = f'{path}{name_file}'
    
    names = [
        'CEP',
        'ENDERECO',
        'COMPLEMENTO',
        'BAIRRO',
        'COD_CIDADE',
        'COD_ESTADO'
    ]

    df = pd.read_csv(file, sep=',', names = names, header=None, index_col = False, encoding='utf-8')
    
    return df

In [5]:
def TransBaseSnow(df):

    df = df.drop(['COD_CIDADE', 'COD_ESTADO'], axis=1)
    df = df.astype(str)
    data_to_insert = [tuple (row) for row in df.values]
    insert_query = '''
        INSERT INTO SDX_GERENCIA_CRESCIMENTO.CONHECIMENTO_CLIENTE_SILVER.B042304_TI_INFORMACOES_CEPS
        (CEP, ENDERECO, COMPLEMENTO, BAIRRO, CIDADE, ESTADO, UF)
        VALUES (%s,%s,%s,%s,%s,%s,%s)
    '''
    cur_snow.executemany(insert_query, data_to_insert)

    update_query = '''
        update SDX_GERENCIA_CRESCIMENTO.CONHECIMENTO_CLIENTE_SILVER.B042304_TI_INFORMACOES_CEPS
        set complemento = null 
        where complemento = 'nan'
    '''

    cur_snow.execute(update_query)
    


In [6]:
def ConcatFiles(path, name_file):
    
    for i, y in enumerate(name_file):
        df = ReadFiles(path, y) 
        df_end_cidades = pd.merge(df, df_cidades[['NOM_CIDADE', 'COD_CIDADE']], on='COD_CIDADE', how='left') 
        df_end_cidades_estados = pd.merge(df_end_cidades, df_estados[['ESTADO', 'UF', 'COD_ESTADO']], on='COD_ESTADO', how='left')   
        TransBaseSnow(df_end_cidades_estados)    
    

In [7]:
print('Iniciando')

path = f'I:/Crescimento/Temporário/Rheydne/CEP/CEPS/'

name_file = os.listdir(path)

ConcatFiles(path, name_file)

print ('Finalizado')

Iniciando
Finalizado
